In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys,os
sys.path.append('..')
from backtester.analysis import *
from backtester.strategy import StrategyBase, OptParam, OptParamArray
from backtester.swarms.ranking import SwarmRanker
from backtester.swarms.rebalancing import SwarmRebalance
from backtester.swarms.filters import SwarmFilter
from backtester.costs import CostsManagerEXOFixed
from backtester.exoinfo import EXOInfo
from backtester.swarms.rankingclasses import *
from backtester.swarms.swarm import Swarm


import pandas as pd
import numpy as np
import scipy

In [ ]:
from scripts.settings import *
try:
    from scripts.settings_local import *
except:
    pass

from exobuilder.data.exostorage import EXOStorage

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)


exo_filter = '*'     # All 
#exo_filter = 'ES_'  # ES only
exo_filter = '*'  # ES Collars (incl Bearish, Bullish, BW and vanilla)

[print(exo) for exo in storage.exo_list(exo_filter)];

# Single EXO analysis

In [ ]:
EXO_NAME = 'NG_BullishCollarBW'

In [ ]:
exo_df, exo_info = storage.load_series(EXO_NAME)

In [ ]:
figsize(15,10)
f, (ax1, ax2, ax3) = plt.subplots(3, gridspec_kw = {'height_ratios':[3, 1, 1]})
exo_df['exo'].plot(ax=ax1);
ax1.set_title('EXO: {0}'.format(EXO_NAME));

exo_df['delta'].plot(ax=ax2);
ax2.set_title('Delta');

exo_df['is_rollover'].plot(ax=ax3);
ax3.set_title('Rollover occured');

## EXO statistics

In [ ]:
stdev_monthly = exo_df['exo'].diff(22).std()
stdev_daily = exo_df['exo'].diff(1).std()
avg_delta = exo_df['delta'].mean()
contracts_executed_per_year = (exo_df['nfutures_executed'].sum() + exo_df['noptions_executed'].sum()) / nyears
nyears = (exo_df.index[-1] - exo_df.index[0]).days / 365.25
rollovers_per_year = exo_df['is_rollover'].sum() / nyears

print("EXO statistics")
print("{0:<20} {1:>10.3}".format('Avg. Delta', avg_delta))
print("{0:<20} {1:>10.1f}".format('Daily StDev', stdev_daily))
print("{0:<20} {1:>10.1f}".format('Monthly StDev', stdev_monthly))
print("{0:<20} {1:>10.1f}".format('Rollovers / yr.', rollovers_per_year))
print("{0:<20} {1:>10.1f}".format('Contracts exec./ yr.', contracts_executed_per_year))

## Volatility analysis

In [ ]:
figsize(10, 5)
f, (ax1, ax2) = plt.subplots(2)

diff1 = exo_df['exo'].diff(1).dropna()
diff20 = exo_df['exo'].diff(20).dropna()

diff1.dropna().hist(bins=50, ax=ax1);
ax1.set_title('1-day changes histogram');


diff20.dropna().hist(bins=50, ax=ax2);
ax2.set_title('20-day changes histogram');


## Delta distribution

In [ ]:
exo_df['delta'].hist(bins=50);